In [3]:
import pyPdf 
import urllib 
import os 
import osgeo.ogr, osgeo.osr 
from osgeo import ogr 
 
file = open('Rangliste.pdf',"r") 
pdf = pyPdf.PdfFileReader(file) 
 
num_pages = pdf.getNumPages() 
states = dict() 
counter = 1 
for i in xrange(num_pages): 
    print "Seite%s"%(i+1) 
    site_content = pdf.getPage(i).extractText() 
    #print site_content 
    if site_content.find('(OECS)') != -1: 
        site_content = site_content.replace('(OECS)','') 
    if site_content.find('Andorra Europa/GUS - 6 D') != -1: 
        site_content = site_content.replace('Andorra Europa/GUS - 6 D','Andorra Europa/GUS (-) 6 D') 
    if site_content.find('Liechtenstein Europa/GUS - 8') != -1: 
        site_content = site_content.replace('Liechtenstein Europa/GUS - 8','Liechtenstein Europa/GUS (-) 8') 
 
 
    lines = site_content.split(")") 

    #print lines 
    #print "a: ", lines[0] 
    cached_lines = [] 
    if i == 0: 
        for x in range(4,(len(lines)-1)): 
            cached_lines.append(lines[x]) 
        lines = cached_lines 
        end = len(lines) 
    else: 
        end = len(lines)-1 


    #print "b: ", lines[0] 
    #print "b: ", lines[len(lines)-1] 

    for j in range(0, end): 
        #if i >=0: print "j(%s): %s"%(j,lines[j]) 
        if j == 0: 
            if i != 0: 
                line = lines[j].replace(str(i+1),'') 
            else: 
                line = lines[j].split(' ') 
                line = line[3]+' '+line[4]+' '+line[5]+' '+line[6] 
        else: 
            line = lines[j] 
        #print line 
        if line.find('Europa/GUS') != -1: 
            line = line.split('Europa/GUS') 
        elif line.find('Naher Osten/Nordafrika') != -1: 
            line = line.split('Naher Osten/Nordafrika') 
        elif line.find('Asien/Pazifik') != -1: 
            line = line.split('Asien/Pazifik') 
        elif line.find('Afrika') != -1: 
            line = line.split('Afrika') 
        elif line.find('Amerika') != -1: 
            line = line.split('Amerika') 
        elif line.find('Ozeanien') != -1: 
            line = line.split('Ozeanien') 
        elif line.find('Organisation Ostkaribischer Staaten (OECS') != -1: 
            line = line.split('Organisation Ostkaribischer Staaten (OECS') 
        elif line.find('Asien') != -1: 
            line = line.split('Asien') 
        line = line[0] 
        #print line 
        if counter <= 9: 
            number = line[1] 
        elif counter > 9 and counter <= 99: 
            number = line[1]+line[2] 
        elif counter > 99: 
            number = line[1]+line[2]+line[3] 
        #print number 
        #if counter > 0: print "counter", counter 
        #	print number 
        line = line.replace(' '+number ,'') 
        cleaned = '' 
        for k in range(1, len(line)-1): 
            cleaned = cleaned + line[k] 
        cleaned = '' 
        if line[0] == ' ': 
            for h in range(1, len(line)-1): 
                cleaned = cleaned + line[h] 
        line = cleaned 

        #rint line 
        states[number] = line 
        #print "line%s: place:%s name:%s"%(j,line[1], line[2]) 
        counter += 1 
 
path='ne_50m_admin_0_countries.shp' 
shapeData = osgeo.ogr.Open(path,1) #the '1' is to make it editable 
layer = shapeData.GetLayerByIndex(0) 
print dir(layer) 
 
#add a new field to the layer 
layer_def = layer.GetLayerDefn() 
print layer_def.GetFieldCount() 
exists = False 
field_name = 'rank' 
for j in xrange(layer_def.GetFieldCount()): 
    if layer_def.GetFieldDefn(j).GetName() == field_name: 
        exists = True 
if exists == False: 
    role_field = ogr.FieldDefn(field_name, ogr.OFTInteger) 
    layer.CreateField(role_field) 
print layer_def.GetFieldCount() 
#********** 
 
#read layer 1st time to set directly findable polygons 
feature = layer.GetNextFeature() 
print dir(feature) 
#print layer.GetFeatureCount() 
found_dict = dict() 
while feature is not None: 
    #print feature.GetField('rank') 
    for i in states: 
        #print states[i] 
        if states[i] == feature.GetField('name'): 
            found_dict[i] = states[i] 
            feature.SetField('rank',int(i)) 
    layer.SetFeature(feature)	 
    feature.Destroy() 
        feature = layer.GetNextFeature() 
 
unfound_states = states 
for i in found_dict: 
    unfound_states.pop(i) 
 
#read layer 2nd time to set the not found polygons 
layer.ResetReading() 
feature = layer.GetNextFeature() 
while feature is not None: 
    if feature.GetField('rank') == None: 
        print feature.GetField('name') 
        print "The rest: %s"%unfound_states 
        print len(unfound_states) 
        print "Cannot find: ",feature.GetField('name') 
        state_rank = raw_input("Number: ") 
        feature.SetField('rank',int(state_rank)) 
        if state_rank != '-1': 
            unfound_states.pop(state_rank) 

    layer.SetFeature(feature) 
    feature.Destroy() 
        feature = layer.GetNextFeature() 


SyntaxError: invalid syntax (<ipython-input-3-192d7d2ba6af>, line 14)